In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
print(train_df.columns)
len(train_df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


12

In [9]:
train_df[(train_df["Age"].isnull()) & (train_df["Cabin"].isnull())]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pima_df = pd.read_csv("data/missing_data_pima-indians-diabetes.csv")

response = "Outcome"
predictors = pima_df.columns.difference([response]).values

# Convert Features to Korean
pima_df.columns = ["임신 횟수", "포도당 농도", "혈압", "피부 두께", "인슐린", "BMI", "당뇨병 내력", "나이", "Outcome"]
pima_df

,임신 횟수,포도당 농도,혈압,피부 두께,인슐린,BMI,당뇨병 내력,나이,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
759,6,190,92,0,0,35.5,0.278,66,1\
760,2,88,58,26,16,28.4,0.766,22,0\
761,9,170,74,31,0,44.0,0.403,43,1\
762,9,89,62,0,0,22.5,0.142,33,0\


In [13]:
# build preprocessing function
def clean_data(df_raw, cols_with_zeros=['포도당 농도', '혈압',
                                       '피부 두께', '인슐린', 'BMI'],
                                       response=['Outcome']):
    df = df_raw.copy()
    
    # replace zero with NaN in features
    df[cols_with_zeros] = df[cols_with_zeros].replace(0, np.nan)
    
    # remove \ and } from response
    df = df.replace(to_replace=r'\\|\}', value='', regex=True)
    
    # change response data type to int
    df[response] = df[response].astype('int')
    
    return df

pima_df_cleaned = clean_data(pima_df)
pima_df_cleaned.head()

,임신 횟수,포도당 농도,혈압,피부 두께,인슐린,BMI,당뇨병 내력,나이,Outcome
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [14]:
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler

# function for KNN model-based imputation of missing values using features without NaN as predictors
def impute_model_basic(df):
    cols_nan = df.columns[df.isna().any()].tolist()  # '포도당 농도', '혈압', '피부 두께', '인슐린', 'BMI'
    cols_no_nan = df.columns.difference(cols_nan).values  # '임신 횟수', '당뇨병 내력', '나이', 'Outcome'
    for col in cols_nan:
        print("cols_nan:", cols_nan, "col:", col)
        test_data = df[df[col].isna()]  # target data (Missing values)
        print("test_data's shape:", test_data.shape)
        train_data = df.dropna()  # training data
        print("training_data's shape:", train_data.shape)
        knr = KNeighborsRegressor(n_neighbors=5).fit(train_data[cols_no_nan], train_data[col])  # .fit(training_data, target_data)
        df.loc[df[col].isna(), col] = knr.predict(test_data[cols_no_nan])
        print("df.loc[df[col].isna(), col]:", df.loc[df[col].isna(), col])
        print("result df:", df)
        print("*" * 40)
    return df

In [4]:
cols_nan = ['포도당 농도', '혈압', '피부 두께', '인슐린', 'BMI']
cols_no_nan = pima_df_cleaned.columns.difference(cols_nan).values

print(cols_nan)
print(cols_no_nan)
print(pima_df_cleaned.isna().sum(), pima_df_cleaned.shape)

['포도당 농도', '혈압', '피부 두께', '인슐린', 'BMI']
['Outcome' '나이' '당뇨병 내력' '임신 횟수']
임신 횟수        0
포도당 농도       5
혈압          35
피부 두께      226
인슐린        371
BMI         11
당뇨병 내력       0
나이           0
Outcome      0
dtype: int64 (764, 9)


In [5]:
impute_model_basic(pima_df_cleaned)

cols_nan: ['포도당 농도', '혈압', '피부 두께', '인슐린', 'BMI'] col: 포도당 농도
test_data's shape: (5, 9)
training_data's shape: (391, 9)
df.loc[df[col].isna(), col]: Series([], Name: 포도당 농도, dtype: float64)
result df:      임신 횟수  포도당 농도    혈압  피부 두께    인슐린   BMI  당뇨병 내력  나이  Outcome
0        6   148.0  72.0   35.0    NaN  33.6   0.627  50        1
1        1    85.0  66.0   29.0    NaN  26.6   0.351  31        0
2        8   183.0  64.0    NaN    NaN  23.3   0.672  32        1
3        1    89.0  66.0   23.0   94.0  28.1   0.167  21        0
4        0   137.0  40.0   35.0  168.0  43.1   2.288  33        1
..     ...     ...   ...    ...    ...   ...     ...  ..      ...
759      6   190.0  92.0    NaN    NaN  35.5   0.278  66        1
760      2    88.0  58.0   26.0   16.0  28.4   0.766  22        0
761      9   170.0  74.0   31.0    NaN  44.0   0.403  43        1
762      9    89.0  62.0    NaN    NaN  22.5   0.142  33        0
763     10   101.0  76.0   48.0  180.0  32.9   0.171  63        0

[764 r

,임신 횟수,포도당 농도,혈압,피부 두께,인슐린,BMI,당뇨병 내력,나이,Outcome
0,6,148.0,72.0,35.0,155.6,33.6,0.627,50,1
1,1,85.0,66.0,29.0,124.6,26.6,0.351,31,0
2,8,183.0,64.0,31.0,175.6,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
759,6,190.0,92.0,30.0,295.2,35.5,0.278,66,1
760,2,88.0,58.0,26.0,16.0,28.4,0.766,22,0
761,9,170.0,74.0,31.0,241.2,44.0,0.403,43,1
762,9,89.0,62.0,34.2,234.0,22.5,0.142,33,0


In [15]:
# function for KNN model-based imputation of missing values using features without NaN as predictors, 
# including progressively added imputed features
def impute_model_progressive(df):
    cols_nan = df.columns[df.isna().any()].tolist()
    cols_no_nan = df.columns.difference(cols_nan).values
    print("----1st col_nan:", cols_nan)
    print("----1st cols_no_nan:", cols_no_nan)
    while len(cols_nan) > 0:
        print("===== exe while statement ======")
        print("length of cols_nan:", len(cols_nan))
        print("*** cols_nan: ***", cols_nan)
        col = cols_nan[0]
        print("col:", col)
        test_data = df[df[col].isna()]
        train_data = df.dropna()
        print("$$ test data's shape:", test_data.shape)
        print("$$ train_data's shape:", train_data.shape)
        knr = KNeighborsRegressor(n_neighbors=5).fit(train_data[cols_no_nan], train_data[col])
        df.loc[df[col].isna(), col] = knr.predict(test_data[cols_no_nan])
        cols_nan = df.columns[df.isna().any()].tolist()
        cols_no_nan = df.columns.difference(cols_nan).values
        print("After cols_nan:", cols_nan)
        print("After cols_no_nan:", cols_no_nan)
        print("*" * 60)
    return df

In [7]:
impute_model_progressive(pima_df_cleaned)

----1st col_nan: []
----1st cols_no_nan: ['BMI' 'Outcome' '나이' '당뇨병 내력' '인슐린' '임신 횟수' '포도당 농도' '피부 두께' '혈압']


,임신 횟수,포도당 농도,혈압,피부 두께,인슐린,BMI,당뇨병 내력,나이,Outcome
0,6,148.0,72.0,35.0,155.6,33.6,0.627,50,1
1,1,85.0,66.0,29.0,124.6,26.6,0.351,31,0
2,8,183.0,64.0,31.0,175.6,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
759,6,190.0,92.0,30.0,295.2,35.5,0.278,66,1
760,2,88.0,58.0,26.0,16.0,28.4,0.766,22,0
761,9,170.0,74.0,31.0,241.2,44.0,0.403,43,1
762,9,89.0,62.0,34.2,234.0,22.5,0.142,33,0


In [16]:
# function for imputing missing data according to a given impute_strategy:
#  drop_rows: drop all rows with one or more missing values
#  drop_cols: drop columns with one or more missing values
#  model_basic: KNN-model-based imputation with fixed predictors
#  model_progressive: KNN-model-based imputation with progressively added predictors
#  mean, median, most_frequent: imputation with mean, median or most frequent values
#
#  cols_to_standardize: if provided, the specified columns are scaled between 0 and 1, after imputation
def impute_data(df_cleaned, impute_strategy=None, cols_to_standardize=None):
    df = df_cleaned.copy()
    if impute_strategy == 'drop_rows':
        df = df.dropna(axis=0)
    elif impute_strategy == 'drop_cols':
        df = df.dropna(axis=1)
    elif impute_strategy == 'model_basic':
        df = impute_model_basic(df)
    elif impute_strategy == 'model_progressive':
        df = impute_model_progressive(df) 
    else:
        arr = SimpleImputer(missing_values=np.nan,strategy=impute_strategy).fit(
            df.values).transform(df.values)
        df = pd.DataFrame(data=arr, index=df.index.values, columns=df.columns.values)
    if cols_to_standardize != None:
        cols_to_standardize = list(set(cols_to_standardize) & set(df.columns.values))
        df[cols_to_standardize] = df[cols_to_standardize].astype('float')
        df[cols_to_standardize] = pd.DataFrame(data=MinMaxScaler().fit(
            df[cols_to_standardize]).transform(df[cols_to_standardize]), 
                                             index=df[cols_to_standardize].index.values,
                                             columns=df[cols_to_standardize].columns.values)
    return df

# Impute_model_basic V.S Impute_model_progressive
Update 차이

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler

def impute_data(df_cleaned, impute_strategy=None, cols_to_standardize=None):
    df = df_cleaned.copy()
    if impute_strategy == 'drop_rows':
        df = df.dropna(axis=0)
    elif impute_strategy == 'drop_cols':
        df = df.dropna(axis=1)
    elif impute_strategy == 'model_basic':
        df = impute_model_basic(df)
    elif impute_strategy == 'model_progressive':
        df = impute_model_progressive(df) 
    else:
        arr = SimpleImputer(missing_values=np.nan,strategy=impute_strategy).fit(
            df.values).transform(df.values)
        df = pd.DataFrame(data=arr, index=df.index.values, columns=df.columns.values)
    if cols_to_standardize != None:
        cols_to_standardize = list(set(cols_to_standardize) & set(df.columns.values))
        df[cols_to_standardize] = df[cols_to_standardize].astype('float')
        df[cols_to_standardize] = pd.DataFrame(data=MinMaxScaler().fit(
            df[cols_to_standardize]).transform(df[cols_to_standardize]), 
                                             index=df[cols_to_standardize].index.values,
                                             columns=df[cols_to_standardize].columns.values)
    return df

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pima_df = pd.read_csv("data/missing_data_pima-indians-diabetes.csv")
response = "Outcome"
predictors = pima_df.columns.difference([response]).values

# Convert Features to Korean
pima_df.columns = ["임신 횟수", "포도당 농도", "혈압", "피부 두께", "인슐린", "BMI", "당뇨병 내력", "나이", "Outcome"]
pima_df

# build preprocessing function
def clean_data(df_raw, cols_with_zeros=['포도당 농도', '혈압',
                                       '피부 두께', '인슐린', 'BMI'],
                                       response=['Outcome']):
    df = df_raw.copy()    
    # replace zero with NaN in features
    df[cols_with_zeros] = df[cols_with_zeros].replace(0, np.nan)    
    # remove \ and } from response
    df = df.replace(to_replace=r'\\|\}', value='', regex=True)    
    # change response data type to int
    df[response] = df[response].astype('int')    
    return df

pima_df_cleaned = clean_data(pima_df)
pima_df_cleaned.head()

,임신 횟수,포도당 농도,혈압,피부 두께,인슐린,BMI,당뇨병 내력,나이,Outcome
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [11]:
cols_to_standardize = None
impute_strategy_list = ['mean', 'median', 'most_frequent', 'constant']
arr = SimpleImputer(missing_values=np.nan, strategy='mean').fit(pima_df_cleaned.values).transform(pima_df_cleaned.values)
arr

array([[6.00e+00, 1.48e+02, 7.20e+01, ..., 6.27e-01, 5.00e+01, 1.00e+00],
       [1.00e+00, 8.50e+01, 6.60e+01, ..., 3.51e-01, 3.10e+01, 0.00e+00],
       [8.00e+00, 1.83e+02, 6.40e+01, ..., 6.72e-01, 3.20e+01, 1.00e+00],
       ...,
       [9.00e+00, 1.70e+02, 7.40e+01, ..., 4.03e-01, 4.30e+01, 1.00e+00],
       [9.00e+00, 8.90e+01, 6.20e+01, ..., 1.42e-01, 3.30e+01, 0.00e+00],
       [1.00e+01, 1.01e+02, 7.60e+01, ..., 1.71e-01, 6.30e+01, 0.00e+00]])

In [66]:
df = pd.DataFrame(data=arr, index=pima_df_cleaned.index.values, columns=pima_df_cleaned.columns.values)
print(df)


     임신 횟수  포도당 농도    혈압      피부 두께         인슐린   BMI  당뇨병 내력    나이  Outcome
0      6.0   148.0  72.0  35.000000  155.659033  33.6   0.627  50.0      1.0
1      1.0    85.0  66.0  29.000000  155.659033  26.6   0.351  31.0      0.0
2      8.0   183.0  64.0  29.165428  155.659033  23.3   0.672  32.0      1.0
3      1.0    89.0  66.0  23.000000   94.000000  28.1   0.167  21.0      0.0
4      0.0   137.0  40.0  35.000000  168.000000  43.1   2.288  33.0      1.0
..     ...     ...   ...        ...         ...   ...     ...   ...      ...
759    6.0   190.0  92.0  29.165428  155.659033  35.5   0.278  66.0      1.0
760    2.0    88.0  58.0  26.000000   16.000000  28.4   0.766  22.0      0.0
761    9.0   170.0  74.0  31.000000  155.659033  44.0   0.403  43.0      1.0
762    9.0    89.0  62.0  29.165428  155.659033  22.5   0.142  33.0      0.0
763   10.0   101.0  76.0  48.000000  180.000000  32.9   0.171  63.0      0.0

[764 rows x 9 columns]


In [72]:
original_columns = pima_df_cleaned.columns.difference(['Outcome'])

In [74]:
original_columns

Index(['BMI', '나이', '당뇨병 내력', '인슐린', '임신 횟수', '포도당 농도', '피부 두께', '혈압'], dtype='object')

In [19]:
impute_data(pima_df_cleaned, "mean", None)

,임신 횟수,포도당 농도,혈압,피부 두께,인슐린,BMI,당뇨병 내력,나이,Outcome
0,6.0,148.0,72.0,35.000000,155.659033,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.000000,155.659033,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,29.165428,155.659033,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.000000,94.000000,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.000000,168.000000,43.1,2.288,33.0,1.0
...,...,...,...,...,...,...,...,...,...
759,6.0,190.0,92.0,29.165428,155.659033,35.5,0.278,66.0,1.0
760,2.0,88.0,58.0,26.000000,16.000000,28.4,0.766,22.0,0.0
761,9.0,170.0,74.0,31.000000,155.659033,44.0,0.403,43.0,1.0
762,9.0,89.0,62.0,29.165428,155.659033,22.5,0.142,33.0,0.0


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from timeit import default_timer as timer

# function for handling missing values 
# and fitting logistic regression on clean data
def logistic_regression(data, impute_strategy=None,
                        cols_to_standardize=None,
                        test_size=0.25,
                        random_state=9001):
    start = timer()
    
    # store original columns
    # ['BMI', '나이', '당뇨병 내력', '인슐린', '임신 횟수', '포도당 농도', '피부 두께', '혈압']
    original_columns = data.columns.difference(['Outcome'])
    df_imputed = impute_data(data, impute_strategy, cols_to_standardize)
    train_data, test_data = train_test_split(df_imputed, test_size=test_size,
                                             random_state=random_state)
    
    # note which predictor columns were dropped or kept
    kept_columns = df_imputed.columns.difference(['Outcome'])
    dropped_columns = original_columns.difference(df_imputed.columns)
    original_columns = original_columns.difference(['Outcome'])
    
    # prepare tensors
    X_train = train_data.drop(columns=['Outcome'])
    y_train = train_data['Outcome']
    X_test = test_data.drop(columns=['Outcome'])
    y_test = test_data['Outcome']
    
    # model training
    logistic_model = LogisticRegressionCV(cv=10, penalty='l2', max_iter=1000).fit(
        X_train, y_train)
    
    # model evaluation
    train_score = accuracy_score(y_train, logistic_model.predict(X_train))
    test_score = accuracy_score(y_test, logistic_model.predict(X_test))
    duration = timer() - start
    print("Classification rate on training data: {}".format(train_score))
    print("Classification rate on test data: {}".format(test_score))
    print("Execution time: {}".format(duration))
    
    return {
        'imputation strategy': impute_strategy,
        'standardized': cols_to_standardize!=None,
        'model': logistic_model,
        'train score': train_score,
        'test score': test_score,
        'execution time (s)': duration
    }

In [24]:
lr_results = []

# prepare data
pima_df_cleaned = clean_data(pima_df)
cols_to_standardize=['나이','BMI','혈압','포도당 농도','인슐린','임신 횟수','피부 두께','당뇨병 내력']

# fit logistic regression for each imputation strategy
# with and without standardizing features
for impute_strategy in ['drop_rows', 'mean', 'model_basic', 'model_progressive']:
    print("impute_strategy:", impute_strategy)
    print("-" * 50)
    print()
    for cols in [None, cols_to_standardize]:
        print("cols:", cols)
        result = logistic_regression(pima_df_cleaned, impute_strategy=impute_strategy, cols_to_standardize=cols)
        print("result:", result)
        lr_results.append(result)
        print("lr_results:", lr_results)
        print("=" * 100)
        print()



impute_strategy: drop_rows
--------------------------------------------------

cols: None
Classification rate on training data: 0.8225255972696246
Classification rate on test data: 0.7040816326530612
Execution time: 1.200236997101456
result: {'imputation strategy': 'drop_rows', 'standardized': False, 'model': LogisticRegressionCV(cv=10, max_iter=1000), 'train score': 0.8225255972696246, 'test score': 0.7040816326530612, 'execution time (s)': 1.200236997101456}
lr_results: [{'imputation strategy': 'drop_rows', 'standardized': False, 'model': LogisticRegressionCV(cv=10, max_iter=1000), 'train score': 0.8225255972696246, 'test score': 0.7040816326530612, 'execution time (s)': 1.200236997101456}]

cols: ['나이', 'BMI', '혈압', '포도당 농도', '인슐린', '임신 횟수', '피부 두께', '당뇨병 내력']
Classification rate on training data: 0.825938566552901
Classification rate on test data: 0.6938775510204082
Execution time: 0.2569465092383325
result: {'imputation strategy': 'drop_rows', 'standardized': True, 'model': Logist

Classification rate on training data: 0.7696335078534031
Classification rate on test data: 0.7801047120418848
Execution time: 1.3281399589031935
result: {'imputation strategy': 'model_basic', 'standardized': False, 'model': LogisticRegressionCV(cv=10, max_iter=1000), 'train score': 0.7696335078534031, 'test score': 0.7801047120418848, 'execution time (s)': 1.3281399589031935}
lr_results: [{'imputation strategy': 'drop_rows', 'standardized': False, 'model': LogisticRegressionCV(cv=10, max_iter=1000), 'train score': 0.8225255972696246, 'test score': 0.7040816326530612, 'execution time (s)': 1.200236997101456}, {'imputation strategy': 'drop_rows', 'standardized': True, 'model': LogisticRegressionCV(cv=10, max_iter=1000), 'train score': 0.825938566552901, 'test score': 0.6938775510204082, 'execution time (s)': 0.2569465092383325}, {'imputation strategy': 'mean', 'standardized': False, 'model': LogisticRegressionCV(cv=10, max_iter=1000), 'train score': 0.7731239092495636, 'test score': 0.75

Classification rate on training data: 0.7818499127399651
Classification rate on test data: 0.7696335078534031
Execution time: 1.3111409307457507
result: {'imputation strategy': 'model_progressive', 'standardized': False, 'model': LogisticRegressionCV(cv=10, max_iter=1000), 'train score': 0.7818499127399651, 'test score': 0.7696335078534031, 'execution time (s)': 1.3111409307457507}
lr_results: [{'imputation strategy': 'drop_rows', 'standardized': False, 'model': LogisticRegressionCV(cv=10, max_iter=1000), 'train score': 0.8225255972696246, 'test score': 0.7040816326530612, 'execution time (s)': 1.200236997101456}, {'imputation strategy': 'drop_rows', 'standardized': True, 'model': LogisticRegressionCV(cv=10, max_iter=1000), 'train score': 0.825938566552901, 'test score': 0.6938775510204082, 'execution time (s)': 0.2569465092383325}, {'imputation strategy': 'mean', 'standardized': False, 'model': LogisticRegressionCV(cv=10, max_iter=1000), 'train score': 0.7731239092495636, 'test score'

In [26]:
# display logistic regression performance
lr_results_df = pd.DataFrame(lr_results)
lr_results_df

,imputation strategy,standardized,model,train score,test score,execution time (s)
0,drop_rows,False,"LogisticRegressionCV(cv=10, max_iter=1000)",0.822526,0.704082,1.200237
1,drop_rows,True,"LogisticRegressionCV(cv=10, max_iter=1000)",0.825939,0.693878,0.256947
2,mean,False,"LogisticRegressionCV(cv=10, max_iter=1000)",0.773124,0.759162,1.204650
3,mean,True,"LogisticRegressionCV(cv=10, max_iter=1000)",0.766143,0.759162,0.270046
4,model_basic,False,"LogisticRegressionCV(cv=10, max_iter=1000)",0.769634,0.780105,1.328140
5,model_basic,True,"LogisticRegressionCV(cv=10, max_iter=1000)",0.771379,0.774869,0.321456
6,model_progressive,False,"LogisticRegressionCV(cv=10, max_iter=1000)",0.781850,0.769634,1.311141
7,model_progressive,True,"LogisticRegressionCV(cv=10, max_iter=1000)",0.787086,0.764398,0.307307


In [ ]:
lr_results_df.drop(['model'], axis=1).drop_duplicates()